In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from importlib.util import find_spec
if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('..')

In [4]:
from qml_hep_lhc.data import ElectronPhoton, MNIST, QuarkGluon
from qml_hep_lhc.data.utils import tf_ds_to_numpy
import argparse
import wandb

import pennylane as qml
import jax.numpy as jnp
import jax
import optax
from jax.nn.initializers import he_uniform
from jax import grad, jit, vmap
from jax import random
import tensorflow_datasets as tfds
from tqdm import tqdm
import numpy as np
import tensorflow as tf

# Added to silence some warnings.
# from jax.config import config
# config.update("jax_enable_x64", True)

import matplotlib.pyplot as plt
import time

In [13]:
args = argparse.Namespace()

# Data
# args.center_crop = 0.2
# args.resize = [8,8]
# args.standardize = 1
# args.power_transform = 1
# args.binary_data = [3,6]
# args.percent_samples = 0.01
# args.processed = 1
args.dataset_type = '1'
# args.labels_to_categorical = 1
args.batch_size = 128
args.validation_split = 0.05

# Base Model
args.wandb = False
args.epochs = 20
args.learning_rate = 0.01

# Quantum CNN Parameters
args.n_layers = 2
args.n_qubits = 3
args.template = 'Qernel'
args.initializer = 'he_uniform'

args.kernel_size = (3,3)
args.strides = (1,1)
args.padding = "VALID"

args.clayer_sizes = [8, 2]

In [14]:
data = QuarkGluon(args)
data.prepare_data()
# data.setup()
# print(data)

In [15]:
data.x_train.shape, data.y_train.shape

((9000, 40, 40, 1), (9000,))

In [16]:
data.x_train = np.fft.fftshift(np.fft.fft2(data.x_train[:,:,:,0]))
data.x_train = np.sqrt(data.x_train.real**2 + data.x_train.imag**2)
data.x_train = data.x_train.reshape(-1,40,40,1)

In [17]:
data.x_test = np.fft.fftshift(np.fft.fft2(data.x_test[:,:,:,0]))
data.x_test = np.sqrt(data.x_test.real**2 + data.x_test.imag**2)
data.x_test = data.x_test.reshape(-1,40,40,1)

In [23]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras import datasets, layers, models

print(tf.__version__)

2.7.0


In [19]:
data.x_train.shape, data.y_train.shape

((9000, 40, 40, 1), (9000,))

In [20]:
data.x_test.shape, data.y_test.shape

((1000, 40, 40, 1), (1000,))

In [31]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(40, 40, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [32]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 38, 38, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 19, 19, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 17, 17, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 6, 6, 64)          36928     
                                                                 
 flatten_2 (Flatten)         (None, 2304)             

In [33]:
model.fit(data.x_train,data.y_train, epochs=100, validation_data=(data.x_test, data.y_test),batch_size = 256)

Epoch 1/100
36/36 [==============================] - 11s 298ms/step - loss: 0.9330 - accuracy: 0.4839 - val_loss: 0.6938 - val_accuracy: 0.5000
Epoch 2/100
36/36 [==============================] - 10s 287ms/step - loss: 0.7194 - accuracy: 0.4972 - val_loss: 0.6985 - val_accuracy: 0.5040
Epoch 3/100
36/36 [==============================] - 10s 288ms/step - loss: 0.6977 - accuracy: 0.5031 - val_loss: 0.6960 - val_accuracy: 0.5000
Epoch 4/100
36/36 [==============================] - 13s 352ms/step - loss: 0.6952 - accuracy: 0.5004 - val_loss: 0.6939 - val_accuracy: 0.4990
Epoch 5/100
36/36 [==============================] - 11s 294ms/step - loss: 0.6967 - accuracy: 0.5002 - val_loss: 0.6936 - val_accuracy: 0.5000
Epoch 6/100
36/36 [==============================] - 11s 295ms/step - loss: 0.6945 - accuracy: 0.5037 - val_loss: 0.6949 - val_accuracy: 0.5000
Epoch 7/100
36/36 [==============================] - 10s 287ms/step - loss: 0.6939 - accuracy: 0.5029 - val_loss: 0.6972 - val_accuracy:

36/36 [==============================] - 11s 292ms/step - loss: 0.4662 - accuracy: 0.7627 - val_loss: 0.9279 - val_accuracy: 0.4920
Epoch 58/100
36/36 [==============================] - 10s 291ms/step - loss: 0.4579 - accuracy: 0.7686 - val_loss: 0.9718 - val_accuracy: 0.4890
Epoch 59/100
36/36 [==============================] - 10s 291ms/step - loss: 0.4511 - accuracy: 0.7724 - val_loss: 0.9407 - val_accuracy: 0.5030
Epoch 60/100
36/36 [==============================] - 11s 291ms/step - loss: 0.4390 - accuracy: 0.7803 - val_loss: 0.9888 - val_accuracy: 0.4870
Epoch 61/100
36/36 [==============================] - 11s 293ms/step - loss: 0.4170 - accuracy: 0.7991 - val_loss: 1.0552 - val_accuracy: 0.5020
Epoch 62/100
36/36 [==============================] - 10s 290ms/step - loss: 0.4093 - accuracy: 0.8051 - val_loss: 1.0539 - val_accuracy: 0.4920
Epoch 63/100
36/36 [==============================] - 10s 289ms/step - loss: 0.4046 - accuracy: 0.8048 - val_loss: 1.0825 - val_accuracy: 0.502